In [1]:
import pandas as pd

In [2]:
raw_df = pd.read_csv('hart_data.csv')
raw_df.head()

,player names,vote-share,position,goals,assists,ops,dps,plus-minus,gps,g-wins,g-losses,gaa,save-percentage,year
0,Wayne Gretzky,38.43,C,51,86,11.3,1.3,14.0,0.0,NaN,NaN,NaN,NaN,1980
1,Marcel Dionne,30.24,C,53,84,11.6,2.0,34.0,0.0,NaN,NaN,NaN,NaN,1980
2,Tony Esposito,11.66,G,0,1,0.0,0.0,0.0,14.0,31.0,22.0,2.98,0.903,1980
3,Guy Lafleur,8.01,RW,50,75,10.8,2.0,40.0,0.0,NaN,NaN,NaN,NaN,1980
4,Danny Gare,3.64,RW,56,33,8.3,2.1,48.0,0.0,NaN,NaN,NaN,NaN,1980


In [3]:
raw_df['mvp'] = pd.Series([0] * raw_df.shape[0])
raw_df.head()

,player names,vote-share,position,goals,assists,ops,dps,plus-minus,gps,g-wins,g-losses,gaa,save-percentage,year,mvp
0,Wayne Gretzky,38.43,C,51,86,11.3,1.3,14.0,0.0,NaN,NaN,NaN,NaN,1980,0
1,Marcel Dionne,30.24,C,53,84,11.6,2.0,34.0,0.0,NaN,NaN,NaN,NaN,1980,0
2,Tony Esposito,11.66,G,0,1,0.0,0.0,0.0,14.0,31.0,22.0,2.98,0.903,1980,0
3,Guy Lafleur,8.01,RW,50,75,10.8,2.0,40.0,0.0,NaN,NaN,NaN,NaN,1980,0
4,Danny Gare,3.64,RW,56,33,8.3,2.1,48.0,0.0,NaN,NaN,NaN,NaN,1980,0


In [4]:
grouped_df = raw_df.groupby('year')

def get_mvp(group):
    winner = group['vote-share'].max()
    group.loc[group['vote-share'] == winner, 'mvp'] = 1
    return group

raw_df = grouped_df.apply(get_mvp, include_groups=False)

In [5]:
raw_df.reset_index(inplace=True)
raw_df = raw_df.drop(columns='level_1')

In [6]:
# split skaters and goalies
skaters = raw_df[raw_df['position'] != 'G']
goalies = raw_df[raw_df['position'] == 'G']

In [7]:
goalies['position'].unique()

array(['G'], dtype=object)

In [8]:
skaters.reset_index(drop=True, inplace=True)
goalies.reset_index(drop=True, inplace=True)

In [9]:
skaters.head()

,year,player names,vote-share,position,goals,assists,ops,dps,plus-minus,gps,g-wins,g-losses,gaa,save-percentage,mvp
0,1980,Wayne Gretzky,38.43,C,51,86,11.3,1.3,14.0,0.0,NaN,NaN,NaN,NaN,1
1,1980,Marcel Dionne,30.24,C,53,84,11.6,2.0,34.0,0.0,NaN,NaN,NaN,NaN,0
2,1980,Guy Lafleur,8.01,RW,50,75,10.8,2.0,40.0,0.0,NaN,NaN,NaN,NaN,0
3,1980,Danny Gare,3.64,RW,56,33,8.3,2.1,48.0,0.0,NaN,NaN,NaN,NaN,0
4,1980,Larry Robinson,1.64,D,14,61,5.5,5.9,38.0,0.0,NaN,NaN,NaN,NaN,0


In [10]:
goalies.head()

,year,player names,vote-share,position,goals,assists,ops,dps,plus-minus,gps,g-wins,g-losses,gaa,save-percentage,mvp
0,1980,Tony Esposito,11.66,G,0,1,0.0,0.0,0.0,14.0,31.0,22.0,2.98,0.903,0
1,1980,Mike Liut,2.55,G,0,0,0.0,0.0,0.0,11.4,32.0,23.0,3.18,0.895,0
2,1981,Mike Liut,41.80,G,0,0,0.0,0.0,0.0,11.6,33.0,14.0,3.35,0.894,0
3,1981,Dan Bouchard,1.06,G,0,4,0.0,0.0,NaN,7.9,23.0,10.0,3.44,0.892,0
4,1981,Mario Lessard,0.35,G,0,1,0.0,0.0,0.0,12.6,35.0,18.0,3.26,0.893,0


In [11]:
# cleaning skater df
skaters = skaters.drop(columns=['gps', 'g-wins', 'g-losses', 'gaa', 'save-percentage'])

In [12]:
skaters['position'].unique()

array(['C', 'RW', 'D', 'LW', 'D/RW', 'F', 'LW/RW'], dtype=object)

In [13]:
skaters[skaters['position'] == 'D/RW']

,year,player names,vote-share,position,goals,assists,ops,dps,plus-minus,mvp
593,2019,Brent Burns,0.29,D/RW,16,67,7.3,4.7,13.0,0


In [14]:
# fix 2019 burns position
skaters.iloc[593, 3] = 'D'

In [15]:
# fix '21, '24 aho position
skaters.iloc[[626, 670], 3] = 'C'

In [16]:
# remove winger distinction
skaters = skaters.replace(['RW', 'LW', 'LW/RW'], 'W')
skaters['position'].unique()

array(['C', 'W', 'D'], dtype=object)

In [17]:
# convert to numerical values
to_integer = ['goals', 'assists', 'plus-minus']
to_float = ['vote-share', 'ops', 'dps']
skaters[to_integer] = skaters[to_integer].astype(int)
skaters[to_float] = skaters[to_float].astype(float)

In [18]:
# one-hot encode position
skaters = pd.get_dummies(skaters, columns=['position'], dtype=int)
skaters.head()

,year,player names,vote-share,goals,assists,ops,dps,plus-minus,mvp,position_C,position_D,position_W
0,1980,Wayne Gretzky,38.43,51,86,11.3,1.3,14,1,1,0,0
1,1980,Marcel Dionne,30.24,53,84,11.6,2.0,34,0,1,0,0
2,1980,Guy Lafleur,8.01,50,75,10.8,2.0,40,0,0,0,1
3,1980,Danny Gare,3.64,56,33,8.3,2.1,48,0,0,0,1
4,1980,Larry Robinson,1.64,14,61,5.5,5.9,38,0,0,1,0


In [19]:
skaters['points'] = skaters['goals'] + skaters['assists']
skaters['tps'] = skaters['ops'] + skaters['dps']
skaters.head()

,year,player names,vote-share,goals,assists,ops,dps,plus-minus,mvp,position_C,position_D,position_W,points,tps
0,1980,Wayne Gretzky,38.43,51,86,11.3,1.3,14,1,1,0,0,137,12.6
1,1980,Marcel Dionne,30.24,53,84,11.6,2.0,34,0,1,0,0,137,13.6
2,1980,Guy Lafleur,8.01,50,75,10.8,2.0,40,0,0,0,1,125,12.8
3,1980,Danny Gare,3.64,56,33,8.3,2.1,48,0,0,0,1,89,10.4
4,1980,Larry Robinson,1.64,14,61,5.5,5.9,38,0,0,1,0,75,11.4


In [20]:
goalies = goalies.drop(columns=['goals', 'assists', 'ops', 'dps', 'plus-minus', 'position'])

In [21]:
goalies[['g-wins', 'g-losses']] = goalies[['g-wins', 'g-losses']].astype(int)

In [22]:
goalies = goalies.drop(columns=['year', 'player names'])
skaters = skaters.drop(columns=['year', 'player names'])

In [23]:
goalies.head()

,vote-share,gps,g-wins,g-losses,gaa,save-percentage,mvp
0,11.66,14.0,31,22,2.98,0.903,0
1,2.55,11.4,32,23,3.18,0.895,0
2,41.80,11.6,33,14,3.35,0.894,0
3,1.06,7.9,23,10,3.44,0.892,0
4,0.35,12.6,35,18,3.26,0.893,0


In [24]:
goalies.to_csv('goalies.csv')
skaters.to_csv('skaters.csv')